In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [3]:
def show_single_image(img_arr):
    # cmap 为颜色图谱，默认为RGB，'binary'为灰度图
    plt.imshow(img_arr, cmap="binary")
    plt.show()

show_single_image(x_train[0])

In [4]:
# 一般参数只需要x_data，y_dataa对应的是类别名的索引，class_names为类别数组
def show_imgs(n_rows, n_cols, x_data, y_data, class_names):
    assert len(x_data) == len(y_data)
    # 方格数要小于数据集中的样本数量
    assert n_rows * n_cols < len(x_data)
    # 定义一张大图
    plt.figure(figsize = (n_cols * 1.4, n_rows * 1.6))
    for row in range(n_rows):
        for col in range(n_cols):
          # 计算图的索引--按行排列
            index = n_cols * row + col
            # 将图分成n_rows * n_cols个小区域， index+1表示在第几个小区域画图--编号按行排列
            plt.subplot(n_rows, n_cols, index+1)
            plt.imshow(x_data[index], cmap="binary",
                       interpolation = 'nearest') 
            # interprolation--缩放图片时插值的方法，'nearest'代表用在最近的元素值来代表中间要插值的小数点的值
            # 去掉坐标系
            plt.axis('off')
            # 给每张子图配上title(标题)
            plt.title(class_names[y_data[index]])
    plt.show()

class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress',
               'Coat', 'Sandal', 'Shirt', 'Sneaker',
               'Bag', 'Ankle boot']
show_imgs(3, 5, x_train, y_train, class_names)


In [15]:
# tf.keras.models.Sequential()

"""
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
"""
# Flatten将28*28的二维矩阵展平为28*28的一维向量
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# relu: y = max(0, x)
# softmax: 将向量变成概率分布. x = [x1, x2, x3], 
#          y = [e^x1/sum, e^x2/sum, e^x3/sum], sum = e^x1 + e^x2 + e^x3

# reason for sparse: y->index. y->one_hot->[]
# "sparse_categorical_crossentropy"为交叉熵的损失函数
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = ["accuracy"])

In [7]:
model.layers

In [16]:
# 打印模型信息 params:参数
model.summary()

In [ ]:
# [None, 784] * W + b -> [None, 300] W.shape [784, 300], b = [300]

In [17]:
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_valid, y_valid))

In [12]:
type(history)

In [13]:
history.history

In [14]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    # 显示网格
    plt.grid(True)
    # 设置坐标轴的范围
    plt.gca().set_ylim(0, 1)
    plt.show()

plot_learning_curves(history)

In [18]:
plot_learning_curves(history)

In [19]:
model.evaluate(x_test, y_test)